# MNIST DATASET

# Defining the model

In [ ]:
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
from argparse import ArgumentParser
import tensorflow_probability as tfp
import copy
import math
class linear_classifier:
    def __init__(self,trainable_weights,trainable_biases):
        self.trainable_weights=tf.Variable(trainable_weights,dtype=tf.float32)
        self.trainable_biases=tf.Variable(trainable_biases,dtype=tf.float32)
        self.optimizer=tf.keras.optimizers.Adam(epsilon=1e-5)   
        self.activation=tfp.bijectors.NormalCDF()
        self.loss=tf.keras.losses.CategoricalCrossentropy()
        self.best_weights=tf.Variable(trainable_weights,dtype=tf.float32)
        self.best_bias=tf.Variable(trainable_biases,dtype=tf.float32)
                                                        	
    @tf.function    
    def	predict(self,input_data):
        x=tf.linalg.matmul(input_data,self.trainable_weights)+self.trainable_biases
        return self.activation(x)
    
    @tf.function
    def update_parameters(self,data,label):
        loss=lambda:self.loss(label,self.predict(data))
        self.optimizer.minimize(loss,[self.trainable_weights,self.trainable_biases])
	
    def train(self,epochs,train_data,train_label,test_data,test_label):
            decline=0	
            prev_acc=0															  
            for ep in range(epochs):
                if decline>8:
                    break
                self.update_parameters(train_data,train_label)
                if ep%100==0:
                    loss,acc=self.test(test_data,test_label)
                    if acc>prev_acc:
                        self.best_weights.assign(self.trainable_weights)
                        self.best_bias.assign(self.trainable_biases)
                    else:
                        decline+=1
                    tf.print("\rloss = {} acc={} ".format(loss,acc),end="")
                    prev_acc=acc

    @tf.function
    def test(self,test_data,test_labels):
            predictions=self.predict(test_data)
            loss=self.loss(predictions,test_labels)
            acc=tf.cast(tf.equal(tf.argmax(predictions,axis=1),tf.argmax(test_labels,axis=1)),dtype=tf.float32)
            return loss,tf.reduce_mean(acc)
            

def init_model_parameters():
	trainable_weights=tf.random.normal(shape=[784,10],mean=0,stddev=1,seed=12)
	trainable_biases=tf.random.normal(shape=[10],mean=0,stddev=1,seed=12)
	return (trainable_weights,trainable_biases)		

if __name__=='__main__':

    (train_data,train_label),(test_data,test_label)=tf.keras.datasets.mnist.load_data()
    (trainable_weights,trainable_biases)=init_model_parameters()
    train_label=tf.one_hot(train_label,depth=10)
    test_label=tf.one_hot(test_label,depth=10)
    train_data=tf.cast(tf.reshape((train_data/255),shape=[-1,28*28]),dtype=tf.float32)
    test_data=tf.cast(tf.reshape((test_data/255),shape=[-1,28*28]),dtype=tf.float32)
    print(train_data.dtype,test_data.dtype,train_label.dtype,test_label.dtype)
    classifier=linear_classifier(trainable_weights,trainable_biases)
    classifier.train(500000,train_data,train_label,test_data,test_label)
    


<dtype: 'float32'> <dtype: 'float32'> <dtype: 'float32'> <dtype: 'float32'>
loss = nan acc=0.09799999743700027 

# **Retrieve best weights**

In [ ]:
classifier.trainable_weights.assign(classifier.best_weights)
classifier.trainable_biases.assign(classifier.best_bias)
tf.print("loss={} acc={}".format(*classifier.test(test_data,test_label)))

loss=1.697241187095642 acc=0.9140999913215637
